In [ ]:
"""
Prepare cav mean and cav std

"""

In [ ]:
import xarray as xr
import summer_paper.useful_functions as uf
from tqdm.notebook import tqdm

import distributed

In [ ]:
client = distributed.Client(n_workers=8,dashboard_address=':8797', local_directory='/tmp', memory_limit='4GB')

READ IN DATA

In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'
to2300 = False

if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_mask='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area']
cell_area_weight = true_grid_cell_area/(4000 * 4000)


In [ ]:

# T and S extrapolated to ice draft depth
T_S_2D_isfdraft = xr.open_mfdataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_*.nc', combine='nested', concat_dim='time').load() #, chunks=({'time': 5}

n = 0
for kisf in tqdm(file_isf.Nisf):
    #conc_isf = uf.choose_isf(file_conc_cut['isfdraft_conc'],isf_stack_mask, kisf)
    T_isf = uf.choose_isf(T_S_2D_isfdraft['theta_in'],isf_stack_mask, kisf)
    S_isf = uf.choose_isf(T_S_2D_isfdraft['salinity_in'],isf_stack_mask, kisf)
    cell_area_kisf = uf.choose_isf(cell_area_weight ,isf_stack_mask, kisf)

    T_mean_cav = uf.weighted_mean(T_isf, 'mask_coord', cell_area_kisf).to_dataset(name='T_mean')
    S_mean_cav = uf.weighted_mean(S_isf, 'mask_coord', cell_area_kisf).to_dataset(name='S_mean')
    T_std_cav = uf.weighted_std(T_isf, 'mask_coord', cell_area_kisf).to_dataset(name='T_std')
    S_std_cav = uf.weighted_std(S_isf, 'mask_coord', cell_area_kisf).to_dataset(name='S_std')

    all_TS_isf = xr.merge([T_mean_cav,S_mean_cav,T_std_cav,S_std_cav])
    all_TS_isf_2D = all_TS_isf.where(T_isf).load()

    if n == 0:
        all_TS_all = all_TS_isf_2D.squeeze().drop('Nisf')
    else:
         all_TS_all =  all_TS_all.combine_first(all_TS_isf_2D).squeeze().drop('Nisf')
    n = n+1

    del all_TS_isf_2D

all_TS_all_unstacked = uf.bring_back_to_2D(all_TS_all)
all_TS_all_unstacked.to_netcdf(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'.nc')

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'.nc')

In [ ]:
print('test')

In [ ]:
all_TS_all_unstacked.to_netcdf(inputpath_profiles+'T_S_2D_meanstd_isf_draft_'+mod+'_'+scenario+'.nc')